<a href="https://colab.research.google.com/github/JGranda11/Proyecto-ai4eng20241/blob/main/05%20-%20preprocesado_con_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamientio con SMOTE
Se trata de hacer un planteamiento de generación de muestras con la libreria SMOTE
SMOTE es una técnica diseñada para abordar este problema generando muestras sintéticas de la clase minoritaria. Funciona seleccionando aleatoriamente muestras de la clase minoritaria y calculando los vecinos más cercanos de cada una de estas muestras. Luego, para cada muestra seleccionada, se selecciona aleatoriamente uno de sus vecinos más cercanos y se crea una nueva muestra sintética a lo largo de la línea que conecta la muestra original y su vecino.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler

## Lectura de los archivos
Habia un preproceado previo ya realizado en los Notebooks 02 y 03, y asignamos estos valores a un nuevo Dataframe

In [3]:
train = pd.read_csv('trainP1.csv')
test = pd.read_csv('testP1.csv')
trainData = train.iloc[:20000]

In [4]:
X = trainData.drop(['RENDIMIENTO_GLOBAL'] , axis=1)
y = trainData["RENDIMIENTO_GLOBAL"].values

Xtest = test.drop('ID', axis=1)

In [5]:
X.shape, y.shape, Xtest.shape

((20000, 56), (20000,), (296786, 56))

In [6]:
from sklearn.model_selection import train_test_split

test_size = 0.3
val_size  = test_size/(1-test_size) # Elementos de validación

print (X.shape, y.shape)
print ("test size %.2f"%test_size)
print ("val size is %.2f (relative to %.2f) "%(val_size, 1-test_size))

#Xtr, ytr son los datos que se usan para entrenar el modelo
#Xts, yts son los datos que usamos para medir una metrica
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=test_size)
print (Xtr.shape, Xts.shape)

(20000, 56) (20000,)
test size 0.30
val size is 0.43 (relative to 0.70) 
(14000, 56) (6000, 56)


## Uso del escalador
En este caso definimos la instancia y las instancias para trabajar en cada modelo

In [8]:
scaler = RobustScaler()

X_train = scaler.fit_transform(Xtr)
X_ts =scaler.fit_transform(Xts)
X_test = scaler.transform(Xtest)

In [7]:
smote = SMOTE(random_state=20)

## Procesado y entrenamiento con SVC

In [ ]:

#X_train, y_train = smote.fit_resample(X_train, y_train)

#modelo = SVC(random_state=20)
#modelo.fit(X_train, y_train)
#y_predition = modelo.predict(X_test)

## Realización del proceso de entrenamiento con RandomForest

In [9]:
from sklearn.ensemble import RandomForestClassifier

X_train, y_train = smote.fit_resample(Xtr, ytr)
X_test, y_test = smote.fit_resample(Xts, yts)
modelo2 = RandomForestClassifier(max_depth=12)
modelo2.fit(X_train, y_train)


RandomForestClassifier(max_depth=12)

In [10]:
modelo2.score(X_train, y_train), modelo2.score(X_train, y_train)

(0.6906003382187148, 0.6906003382187148)

## Realizacion con DecisionTree

In [11]:
from sklearn.ensemble import DecisionTreeClassifier

X_train, y_train = smote.fit_resample(X_train, y_train)

modelo3 = DecisionTreeClassifier(max_depth=12)
modelo3.fit(X_train, y_train)
y_predition3 = modelo3.predict(X_test)

In [12]:
modelo3.score(X_train, y_train), modelo3.score(X_train, y_train)

(0.5614430665163472, 0.5614430665163472)

## Anotaciones de Score

Aunque vemos muy buenos puntajes, los modelos entrenaban con demasiado tiempo, a veces colapsaba la RAM y cuando lograban realizar la predicción no era lo suficientemente buen en Kaggle


In [ ]:
#modelo.score(X_train, y_train)

0.4726878327746007

In [13]:
modelo2.score(X_train, y_train)

0.6906003382187148

In [15]:
modelo3.score(X_train, y_train)

0.5614430665163472

In [18]:
y_prediction2 = modelo2.predict(X_test)

In [19]:
y_prediction2

array([3., 2., 3., ..., 0., 0., 2.])

In [20]:
y_prediction3 = modelo3.predict(X_test)

In [21]:
y_prediction3

array([3., 2., 3., ..., 2., 0., 3.])

## Generación de las predicciones
Como el `modelo2` es el que tiene mejor comportamiento realizamos el envio a Kaggle con esa validación

In [22]:
idTest = pd.DataFrame(test["ID"])

Reference = ['bajo', 'medio-bajo', 'medio-alto', 'alto']
y_preditionReference = [Reference[int(i)] for i in y_prediction2]

idTest['RENDIMIENTO_GLOBAL'] = pd.Series(y_preditionReference, name='RENDIMIENTO_GLOBAL')

In [23]:
 idTest.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,alto
4,785185,bajo


In [24]:
idTest.to_csv('submission3.csv', index=False)

In [25]:
!head submission3.csv

ID,RENDIMIENTO_GLOBAL
550236,alto
98545,medio-alto
499179,alto
782980,alto
785185,bajo
58495,alto
705444,alto
557548,bajo
519909,alto
